In [17]:
import pandas as pd

In [18]:
import numpy as np

In [19]:
import json
from pandas.io.json import json_normalize

In [8]:
# load json as string
sample_json_s = json.load((open('data/world_bank_projects.json')))
sample_json_n = json_normalize(sample_json_s)
sample_json_n.head(2)

,_id.$oid,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,source,status,supplementprojectflg,theme1.Name,theme1.Percent,theme_namecode,themecode,totalamt,totalcommamt,url
0,52b213b38594d8a2be17c780,1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,IBRD,Active,N,Education for all,100,"[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,52b213b38594d8a2be17c781,2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,IBRD,Active,N,Other economic management,30,"[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


In [9]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects.json')
sample_json_df.head(2)

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,

* Find the 10 countries with most projects
* Find the top 10 major project themes (using column 'mjtheme_namecode')
* In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

## Answer 1

In [6]:
s = sample_json_df.countryshortname
#s[s.isnull()]
#s.head()
s.value_counts().head(10)
#d = s.value_counts()[:10]
#d = s.value_counts()[0:10]
#d

Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Nepal                 12
Bangladesh            12
Morocco               12
Mozambique            11
Africa                11
Name: countryshortname, dtype: int64

In [7]:
a = sample_json_df[['countryshortname', 'project_name']]
a.head()

,countryshortname,project_name
0,Ethiopia,Ethiopia General Education Quality Improvement...
1,Tunisia,TN: DTF Social Protection Reforms Support
2,Tuvalu,Tuvalu Aviation Investment Project - Additiona...
3,"Yemen, Republic of",Gov't and Civil Society Organization Partnership
4,Lesotho,Second Private Sector Competitiveness and Econ...


In [8]:
b = a.groupby('countryshortname').agg([pd.Series.nunique])
b.sort_values(by = [('project_name', 'nunique')], ascending = False).head(10)

,project_name
,nunique
countryshortname,
China,19
Indonesia,19
Vietnam,17
India,16
"Yemen, Republic of",13
Nepal,12
Bangladesh,12
Morocco,12


## Answer 2

In [52]:
for r in themename.iterrows(): print(r[1][0], r[1][1])

8 Human development
11 
1 Economic management
6 Social protection and risk management
5 Trade and integration
2 Public sector governance
11 Environment and natural resources management
6 Social protection and risk management
7 Social dev/gender/inclusion
7 Social dev/gender/inclusion
5 Trade and integration
4 Financial and private sector development
6 Social protection and risk management
6 
2 Public sector governance
4 Financial and private sector development
11 Environment and natural resources management
8 
10 Rural development
7 
2 Public sector governance
2 Public sector governance
2 Public sector governance
10 Rural development
2 
10 Rural development
6 Social protection and risk management
6 Social protection and risk management
11 Environment and natural resources management
4 
5 Trade and integration
11 Environment and natural resources management
10 Rural development
6 Social protection and risk management
6 Social protection and risk management
6 Social protection and risk m

In [53]:
for index, row in themename.iterrows(): print(row['code'], row['name'])

8 Human development
11 
1 Economic management
6 Social protection and risk management
5 Trade and integration
2 Public sector governance
11 Environment and natural resources management
6 Social protection and risk management
7 Social dev/gender/inclusion
7 Social dev/gender/inclusion
5 Trade and integration
4 Financial and private sector development
6 Social protection and risk management
6 
2 Public sector governance
4 Financial and private sector development
11 Environment and natural resources management
8 
10 Rural development
7 
2 Public sector governance
2 Public sector governance
2 Public sector governance
10 Rural development
2 
10 Rural development
6 Social protection and risk management
6 Social protection and risk management
11 Environment and natural resources management
4 
5 Trade and integration
11 Environment and natural resources management
10 Rural development
6 Social protection and risk management
6 Social protection and risk management
6 Social protection and risk m

In [97]:
dj = pd.DataFrame()
for s in sample_json_df.mjtheme_namecode:
    for r in s:
        j = json_normalize(r)
        dj = pd.concat([dj, j])
djl = dj.groupby(['name','code'])['code','name'].count().sort_values('code',ascending=False).drop('name',1).rename(columns={'code':'count'}).reset_index('name')
djl.head()

,name,count
code,,
11,Environment and natural resources management,223
10,Rural development,202
8,Human development,197
2,Public sector governance,184
6,Social protection and risk management,158


In [87]:
#djl = djl[djl['name'] != ""] #remove rows with empty project name
djs = djl[djl.name != ""]
djs.head(10)

,name,count
code,,
11,Environment and natural resources management,446
10,Rural development,404
8,Human development,394
2,Public sector governance,368
6,Social protection and risk management,316
4,Financial and private sector development,260
7,Social dev/gender/inclusion,238
5,Trade and integration,144
9,Urban development,94


## Answer 3

In [98]:
djl #from Answer 2

,name,count
code,,
11,Environment and natural resources management,223
10,Rural development,202
8,Human development,197
2,Public sector governance,184
6,Social protection and risk management,158
4,Financial and private sector development,130
7,Social dev/gender/inclusion,119
5,Trade and integration,72
9,Urban development,47
